In [3]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
#import root_numpy as rtnpy
#from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import yoda

In [4]:
import sys
sys.path.insert(0,'../test/')
from DrawHistogram import *
from Samples_dijetSel import checkDict, dictSamples
sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrstyle
from variables_dijetSel import nSubVariables as nSubVariables_dijetSel


In [5]:

canvas = {}
textBox=ROOT.TLatex()
textBox.SetTextSize(0.10)
textBox.SetTextAlign(12)


### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [6]:
def Fill1D(h0,h,bins):
    N=h0.GetNbinsX()
    content=np.zeros((len(bins)-1),dtype=float)
    errors=np.zeros((len(bins)-1),dtype=float)

    for i in range(len(bins)-1):
        content[i]=0
    for i in range(N):
        center=h0.GetBinCenter(i+1)
        index=np.digitize(np.array([center]),bins)
        if index[0] < len(bins):
            content[index[0]-1]+=h0.GetBinContent(i+1)
            errors[index[0]-1]+=h0.GetBinError(i+1)
    for i in range(len(bins)-1):
        h.SetBinContent(i+1,content[i])
        h.SetBinError(i+1,errors[i])

def Fill2D(h0,h,binx,biny):
    Nx=h0.GetNbinsX()
    Ny=h0.GetNbinsY()
    hx=h0.ProjectionX()
    hy=h0.ProjectionY()
    content=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    errors=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            content[i][j]=0
    for i in range(Nx):
        for j in range(Ny):
            centerx=hx.GetBinCenter(i+1)
            indexx=np.digitize(np.array([centerx]),binx)
            centery=hy.GetBinCenter(j+1)
            indexy=np.digitize(np.array([centery]),biny)
            if indexx[0] < len(binx) and indexy[0] < len(biny):
                content[indexx[0]-1][indexy[0]-1]+=h0.GetBinContent(i+1,j+1)
                errors[indexx[0]-1][indexy[0]-1]+=ROOT.TMath.Power(h0.GetBinError(i+1,j+1),2)
                
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            h.SetBinContent(i+1,j+1,content[i][j])
            h.SetBinError(i+1,j+1,ROOT.TMath.Sqrt(errors[i][j]))


In [7]:
def DoUnfolding(Response,Reco):
    tunfolder = ROOT.TUnfoldDensity(Response,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfold.kRegModeCurvature, 
                                    ROOT.TUnfold.kEConstraintNone, 
                                    ROOT.TUnfoldDensity.kDensityModeBinWidth)
    tunfolder.SetInput(Reco)
    tunfolder.DoUnfold(0.)
    return tunfolder.GetOutput("MC_unfolded")

def CrossClosure(response1,reco1,response2,reco2):
    unf11=DoUnfolding(response1,reco1)
    unf12=DoUnfolding(response2,reco1)
    unf21=DoUnfolding(response1,reco2)
    unf22=DoUnfolding(response2,reco2)
    return unf11,unf12,unf21,unf22

def SelfClosure(response1,reco2,response2,reco1):
    unf21=DoUnfolding(response1,reco2)
    unf12=DoUnfolding(response2,reco1)
    return unf21,unf12

In [8]:
def getRebinnedRescaledHist(hist_name, file_name, xrange, scale, isMC=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(scale)
    
    return newHist

In [9]:
from array import array
ROOT.gStyle.SetOptStat(0)

In [10]:
##########################################################################
def loadHistograms( samples, var, sel, sysUnc=[], isMC=True, addGenInfo=True, respOnly=False, lumi=1., variables={}, year='2017', process='data' ):
    """docstring for loadHistograms"""

    if sysUnc==[]: SYSUNC = [ '_nom' ] 
    else: SYSUNC = [ s+u for u in ['Up', 'Down'] for s in sysUnc if not s.startswith(('_model', '_hdamp', '_Tune', '_CR', '_erdON', '_mtop')) ]
    flip = False
    
    allHistos = {}
    for isam in samples:
        if sysUnc!=[]:
            for i in ([ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal' ]):
                if i in isam:
                    flip=True
                    tmpSYSUNC = [i+u for u in ['Up','Down']]
                    continue
                    
        if not flip: tmpList = [ 'reco'+var+syst+sel for syst in SYSUNC]
        else: tmpList = ['reco'+var+syst+sel for syst in tmpSYSUNC]
        #print (tmpList)
        #if isMC and addGenInfo: tmpList = tmpList + [ 'gen'+var+sel ] + [ 'resp'+var+syst+sel for syst in SYSUNC ]
        if isMC and addGenInfo and not flip: tmpList = tmpList + [ 'gen'+var+sel] + [ 'resp'+var+syst+sel for syst in SYSUNC]
        
        elif isMC and addGenInfo and flip: 
            tmpList = tmpList + [ 'gen'+var+sel]
            for syst in tmpSYSUNC:
                tmpList = tmpList + ['resp'+var+syst+sel]
        
        if respOnly and not flip: 
            tmpList = [ 'resp'+var+syst+sel for syst in SYSUNC] 
            #print (SYSUNC,tmpList)
        elif respOnly and flip: 
            tmpList = ['resp'+var+syst+sel for syst in tmpSYSUNC ] #+ [ 'missgen'+var+sel ]
            #print (tmpSYSUNC,tmpList)
        
        for ih in tmpList:
            #print ('Processing '+isam+' '+ih, flip)
            #print (samples[isam][0])
            if isMC:
                allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
                tmpIsam = 'TT' if isam.startswith('data') else isam
                MCScale = samples[isam][1]['XS'] * lumi / samples[isam][1][year][('nGenWeights' if process.startswith('data') else 'nGenWeights') ]
                allHistos[isam+'_'+ih].Scale( MCScale )
            else:
                if sel.startswith('_dijet') and process.startswith('data'):
                    tmpdataHistos = {}
                    for it in checkDict( 'JetHT', dictSamples )[year]['triggerList']:
                        tmpdataHistos[ it ] = samples[isam][0].Get( 'jetObservables/'+ih.replace( sel, '_'+it+sel ) )
                        tmpdataHistos[ it ].Scale( checkDict( 'JetHT', dictSamples )[year]['triggerList'][it] )
                    allHistos[ isam+'_'+ih ] = tmpdataHistos[next(iter(tmpdataHistos))].Clone()
                    allHistos[ isam+'_'+ih ].Reset()
                    for i in tmpdataHistos: allHistos[isam+'_'+ih].Add( tmpdataHistos[i] )
                else:
                    allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )

            if len(variables[var]['bins'])==1:
                genBin = variables[var]['bins'][0]
                recoBin = variables[var]['bins'][0]/2
            else:
                genBin = variables[var]['bins']
                recoBin = np.sort(np.append( variables[var]['bins'], np.array([ (variables[var]['bins'][i]+variables[var]['bins'][i+1])/2 for i in range(len(variables[var]['bins'])-1) ]  ) ))

            if not ih.startswith('resp'):
                if len(variables[var]['bins'])==1:
                    if ih.startswith(('reco','fake', 'true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'].Rebin( genBin )
                        allHistos[isam+'_'+ih].Rebin( recoBin )
                    else: allHistos[isam+'_'+ih].Rebin( genBin )
                else:
                    if ih.startswith(('reco','fake','true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih+'_genBin'].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin_genBin", array( 'd', genBin ) )
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(recoBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', recoBin ) )
                    else:
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', genBin ) )
            else:
                if len(variables[var]['bins'])==1: allHistos[isam+'_'+ih].Rebin2D( genBin, recoBin )
                else:

                    #### fancy way to create variable binning TH2D
                    tmpHisto = ROOT.TH2F( allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", len(genBin)-1, array( 'd', genBin), len(recoBin)-1, array( 'd', recoBin) )

                    tmpArrayContent = np.zeros((len(genBin), len(recoBin)))
                    tmpArrayError = np.zeros((len(genBin), len(recoBin)))

#                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
#                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
#                            tmpHisto.Fill( allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx), allHistos[isam+'_'+ih].GetYaxis().GetBinCenter(biny), allHistos[isam+'_'+ih].GetBinContent( binx, biny ) )

                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
                        by = allHistos[isam+'_'+ih].GetYaxis().GetBinCenter( biny )
                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
                            bx = allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx)
                            for iX in range( len(genBin)-1 ):
                                for iY in range( len(recoBin)-1 ):
                                    if (bx<genBin[iX+1] and bx>genBin[iX]) and (by<recoBin[iY+1] and by>recoBin[iY]):
                                        jbin = allHistos[isam+'_'+ih].GetBin(binx,biny)
                                        tmpArrayContent[iX][iY] = tmpArrayContent[iX][iY] + allHistos[isam+'_'+ih].GetBinContent( jbin )
                                        tmpArrayError[iX][iY] = tmpArrayError[iX][iY] + ROOT.TMath.Power( allHistos[isam+'_'+ih].GetBinError( jbin ), 2 )

                    for biny in range( 1, tmpHisto.GetNbinsY()+1 ):
                        for binx in range( 1, tmpHisto.GetNbinsX()+1 ):
                            #if (binx <= ((biny+1)/2.)+4) and (binx >= ((biny+1)/2.)-4):
                            tmpHisto.SetBinContent( tmpHisto.GetBin(binx,biny), tmpArrayContent[binx-1][biny-1] )
                            tmpHisto.SetBinError( tmpHisto.GetBin(binx,biny), ROOT.TMath.Sqrt(tmpArrayError[binx-1][biny-1] ) )

                    tmpHisto.Sumw2()
                    allHistos[isam+'_'+ih] = tmpHisto

                #if isMC: allHistos[isam+'_'+ih].Scale( MCScale )
                ##### For tests, projections directly from 2D
                allHistos[isam+'_genJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionY()
                allHistos[isam+'_recoJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionX()


    tmpHistos = { k:v for (k,v) in allHistos.items() if 'Inf' in k }
    for ih in tmpHistos:
        for jh in allHistos:
            if (jh.endswith('0'+ih.split('Inf')[1])) and not ('Inf' in jh ):
                tmpHistos[ih].Add( allHistos[jh].Clone() )
    if len(tmpHistos)>0: allHistos = tmpHistos

    return allHistos

In [11]:
def drawUnfold( ivar, selection, process, year, dataJetHisto, genJetHisto, unfoldHisto, unfoldHistowoUnc, altMCHisto,
               foldHisto, recoJetHisto, ratioUncHisto, ratiototUncHisto, ratiosystUncHisto, labelX, maxX, tlegendAlignment, outputName ):
    """docstring for drawUnfold"""
    #print ("Drawing unfolding")
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    can = ROOT.TCanvas('can'+ivar, 'can'+ivar,  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1"+ivar, "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2"+ivar, "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()

    pad1.cd()

    if tlegendAlignment.startswith('right'): legend=ROOT.TLegend(0.65,0.65,0.90,0.88)

    else: legend=ROOT.TLegend(0.20,0.65,0.40,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)

    dataIntegral = dataJetHisto.Integral()
    unfoldIntegral = unfoldHisto.Integral()
    #tmpMax = float(unfoldHisto.GetMaximum())
    
    #unfoldHisto.Scale(1, 'width')  ### divide by bin width
    unfoldHisto.Scale(dataIntegral/unfoldHisto.Integral(), 'width')  ### divide by bin width
    unfoldHisto.SetMarkerStyle(8)
    #unfoldHisto.SetMarkerSize(2)
    unfoldHisto.SetMarkerColor(ROOT.kBlack)
    unfoldHisto.SetLineColor(ROOT.kBlack)
    legend.AddEntry( unfoldHisto, 'Data', 'pe' )
    
    #genJetHisto.Scale(1, 'width')
    #genJetHisto.Scale(scaleFactor)
    genJetHisto.Scale(dataIntegral/genJetHisto.Integral(), 'width')  ### divide by bin width
    genJetHisto.SetLineWidth(2)
    genJetHisto.SetLineColor(ROOT.kRed)
    genJetHisto.SetMarkerColor(ROOT.kRed)
    genJetHisto.SetMarkerStyle(25)
    legend.AddEntry( genJetHisto, 'MG5+Pythia8', 'lp' )
    
    altMCHisto.Scale(dataIntegral/altMCHisto.Integral(), 'width')  ### divide by bin width
    altMCHisto.SetLineWidth(2)
    altMCHisto.SetLineColor(ROOT.kBlue)
    altMCHisto.SetMarkerColor(ROOT.kBlue)
    altMCHisto.SetMarkerStyle(25)
    legend.AddEntry( altMCHisto, 'Herwig7', 'lp' )

    

    unfoldHisto.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d#'+labelX.split('#')[1]+'}' )
    #unfoldHisto.GetYaxis().SetTitleOffset(0.95)
    unfoldHisto.GetYaxis().SetTitleSize(0.05)
    unfoldHisto.SetMaximum( 1.5*max([ genJetHisto.GetMaximum(), unfoldHisto.GetMaximum()] )  )

    unfoldHisto.Draw( "E")
    genJetHisto.Draw( "histe same")
    altMCHisto.Draw("histe same")
    #unfoldHistowoUnc.Draw( "e1 same")
    #foldHisto.Draw( "histe same")
    #if selfClosure: foldHisto.Draw( "histe same")
    #dataJetHisto.Draw( "histe same")
    #recoJetHisto.Draw( "histe same")
    #if process.startswith('data'): recoJetHisto.Draw( "hist same")
    
    selText = textBox.Clone()
    selText.SetTextFont(42)
    selText.SetTextSize(0.045)

    selText.SetNDC()

    if selection.startswith("_dijet"): seltext = ( 'Central' if 'Central' in labelX  else 'Outer' )+' dijet region'
    elif selection.startswith("_W"): seltext = ' Boosted W region'
    elif selection.startswith("_top"): seltext = ' Boosted top region'
    selText.DrawLatex( ( 0.2 if tlegendAlignment.startswith('right') else 0.7 ), 0.87, seltext )

    
    legend.Draw()
    if process.startswith('data'):
        CMS_lumi.extraText = "Preliminary"
        CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 13 TeV"+('' if year.startswith('all') else ", "+( '2017+2018' if year.startswith('all') else year ) )
    else:
        CMS_lumi.extraText = "Simulation Preliminary"
        CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
    CMS_lumi.relPosX = 0.12
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    pad2.SetGrid()
    pad2.SetTopMargin(0.)
    pad2.SetBottomMargin(0.3)
    tmpPad2= pad2.DrawFrame( 0, 0., maxX, 1.9 )
    tmpPad2.GetXaxis().SetTitle( labelX )
    tmpPad2.GetYaxis().SetTitle( "Sim./Data" )
    tmpPad2.GetYaxis().SetTitleOffset( 0.5 )
    tmpPad2.GetYaxis().SetRangeUser(-2.5, 4.5 )
    tmpPad2.GetYaxis().CenterTitle()
    tmpPad2.SetLabelSize(0.12, 'x')
    tmpPad2.SetTitleSize(0.12, 'x')
    tmpPad2.SetLabelSize(0.12, 'y')
    tmpPad2.SetTitleSize(0.12, 'y')
    tmpPad2.SetNdivisions(505, 'x')
    tmpPad2.SetNdivisions(505, 'y')
    pad2.Modified()
    
    hRatioUp = ROOT.TGraphAsymmErrors()
    hRatioUp.Divide( genJetHisto, unfoldHisto, 'pois' )
    hRatioUp.SetLineColor(ROOT.kRed)
    hRatioUp.SetMarkerColor(ROOT.kRed)
    #hRatioUp.SetLineWidth(2)
    hRatioUp.SetMarkerStyle(25)
    hRatioUp.Draw('P0')
    
    hRatioUp2 = ROOT.TGraphAsymmErrors()
    hRatioUp2.Divide( altMCHisto, unfoldHisto, 'pois' )
    hRatioUp2.SetLineColor(ROOT.kBlue)
    hRatioUp2.SetMarkerColor(ROOT.kBlue)
    #hRatioUp.SetLineWidth(2)
    hRatioUp2.SetMarkerStyle(25)
    hRatioUp2.Draw('P0 same')
    
    #hRatioDown.Draw('P same')
    ratioUncHisto.SetFillColor(15)
    #ratioUncHisto.SetLineColor(0)
    ratioUncHisto.SetFillStyle(3005)
    ratioUncHisto.SetMarkerStyle(0)
    #ratioUncHisto.Scale(1,'width')
    ratioUncHisto.Draw('E3 same')

    ratiototUncHisto.SetFillColor(8)
    #ratiototUncHisto.SetLineColor(0)
    ratiototUncHisto.SetFillStyle(3004)
    ratiototUncHisto.SetMarkerStyle(0)
    #ratiototUncHisto.Scale(1,'width')    
    ratiototUncHisto.Draw('E3 same')
    
    ratiosystUncHisto.SetFillColor(9)
    #ratiosystUncHisto.SetLineColor(0)
    ratiosystUncHisto.SetFillStyle(3006)
    ratiosystUncHisto.SetMarkerStyle(0)
    #ratiosystUncHisto.Scale(1,'width')    
    ratiosystUncHisto.Draw('E3 same')

    
    ratioLegend=ROOT.TLegend(0.20,0.85,0.80,0.95)
    ratioLegend.SetTextSize(0.095)
    ratioLegend.SetNColumns(3)
    ratioLegend.SetFillColorAlpha(10,0.75)
    ratioLegend.SetBorderSize(0)
    #ratioLegend.SetTextSize(0.1)
    ratioLegend.AddEntry( ratiototUncHisto, 'Total', 'f' )
    ratioLegend.AddEntry( ratioUncHisto, 'Stat.', 'f' )
    ratioLegend.AddEntry( ratiosystUncHisto, 'Syst.', 'f' )
    ratioLegend.Draw()
    png = outputName.split('.pdf')[0]+'.png'
    can.SaveAs(outputName)
    can.SaveAs(png)
    ROOT.gStyle.SetPadRightMargin(0.09)     ## reseating
    ROOT.gStyle.SetPadLeftMargin(0.12)

In [12]:
def drawUncertainties( ivar, unfoldHistoTotUnc, uncerUnfoldHisto, labelX, tlegendAlignment, outputName,year ):
    """docstring for drawUncUncertainties"""
    colors = [ 2, 3, 4, 6, 7, 8, 9, 50, 205, 225, 94, 221, 92, 16, 28, 219, 225, 128,]
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    canUnc = ROOT.TCanvas('canUnc'+ivar, 'canUnc'+ivar,  10, 10, 750, 500 )
    
    if tlegendAlignment.startswith('right'): legend=ROOT.TLegend(0.65,0.65,0.90,0.88)
    else: legend=ROOT.TLegend(0.20,0.65,0.40,0.88)
    legend.SetFillStyle(0)
    legend.SetNColumns(2)
    legend.SetTextSize(0.02)
    legend.SetBorderSize(0)
    uncScaleFactor = 1/uncerUnfoldHisto[ivar+'_TotalUnc'].Integral()
    print ("unc. scale factor", uncScaleFactor)

    uncerUnfoldHisto[ivar+'_StatTotal'].SetLineWidth(2)
    uncerUnfoldHisto[ivar+'_StatTotal'].SetLineStyle(2)
    uncerUnfoldHisto[ivar+'_StatTotal'].Scale( uncScaleFactor )
    uncerUnfoldHisto[ivar+'_StatTotal'].GetXaxis().SetTitle(labelX)
    uncerUnfoldHisto[ivar+'_StatTotal'].GetYaxis().SetTitle('Fractional Uncertainty')
    uncerUnfoldHisto[ivar+'_StatTotal'].SetMaximum(5)
    uncerUnfoldHisto[ivar+'_StatTotal'].GetYaxis().SetRangeUser(5*(10**(-4)),5)
    uncerUnfoldHisto[ivar+'_StatTotal'].SetLineColor(88)
    uncerUnfoldHisto[ivar+'_StatTotal'].Draw('hist ')
    
    
    uncerUnfoldHisto[ivar+'_TotalUnc'].SetLineWidth(2)
    uncerUnfoldHisto[ivar+'_TotalUnc'].SetLineStyle(2)
    uncerUnfoldHisto[ivar+'_TotalUnc'].Scale( uncScaleFactor )
    uncerUnfoldHisto[ivar+'_TotalUnc'].GetXaxis().SetTitle(labelX)
    uncerUnfoldHisto[ivar+'_TotalUnc'].GetYaxis().SetTitle('Fractional Uncertainty')
    uncerUnfoldHisto[ivar+'_TotalUnc'].SetMarkerStyle(4)
    uncerUnfoldHisto[ivar+'_TotalUnc'].SetLineColor(1)
    legend.AddEntry( uncerUnfoldHisto[ivar+'_TotalUnc'], 'Total Unc', 'l' )   
    legend.AddEntry( uncerUnfoldHisto[ivar+'_StatTotal'], 'Stat. Tot.', 'l' )    
    uncerUnfoldHisto[ivar+'_TotalUnc'].Draw('hist same')
    
    uncerUnfoldHisto[ivar+'_SystTotal'].SetLineWidth(2)
    uncerUnfoldHisto[ivar+'_SystTotal'].SetLineStyle(2)
    uncerUnfoldHisto[ivar+'_SystTotal'].Scale( uncScaleFactor )
    uncerUnfoldHisto[ivar+'_SystTotal'].SetLineColor(209)
    legend.AddEntry( uncerUnfoldHisto[ivar+'_SystTotal'], 'Syst. Tot.', 'l' )    
    uncerUnfoldHisto[ivar+'_SystTotal'].Draw('hist same')
    
    
    dummy=0
    #print (uncerUnfoldHisto.keys())
    for k in uncerUnfoldHisto:
        if ('Bkg' in k or k.endswith('Total')) and not k.endswith(('TotalUnc', 'SystTotal', 'StatTotal')) and not 'CM' in k:
            legend.AddEntry( uncerUnfoldHisto[k], k.split('_')[-1].split('Total')[0].replace('TOTAL', '').replace('WEIGHT', ''), 'l' )
            text = k.split('_')[-1].split('Total')[0].replace('TOTAL', '').replace('WEIGHT', '')
            if text.startswith(('ISR', 'FSR', 'JER', 'JES', 'PU', 'PDF', 'Bkg')):
                uncerUnfoldHisto[k].SetLineStyle(2)
            uncerUnfoldHisto[k].SetLineColor(colors[dummy])
            uncerUnfoldHisto[k].SetLineWidth(2)
            uncerUnfoldHisto[k].Scale( uncScaleFactor )
            uncerUnfoldHisto[k].Draw("hist same")
            dummy=dummy+1
    
    dummy = dummy - 5
    for k in uncerUnfoldHisto:
        if ('CM' in k and k.endswith('Total')):
            legend.AddEntry( uncerUnfoldHisto[k], k.split('_')[-1].split('Total')[0].replace('TOTAL', '').replace('WEIGHT', ''), 'l' )
            uncerUnfoldHisto[k].SetLineStyle(3)
            uncerUnfoldHisto[k].SetLineColor(colors[dummy])
            uncerUnfoldHisto[k].SetLineWidth(1)
            uncerUnfoldHisto[k].Scale( uncScaleFactor )
            uncerUnfoldHisto[k].Draw("hist same")
            dummy=dummy+1

   
    
    legend.Draw()
    CMS_lumi.extraText = "Simulation Preliminary"
    CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(canUnc, 4, 0)
    canUnc.SetLogy()
    canUnc.Update()

    canUnc.SaveAs(outputName)
    
    

In [13]:
def draw2D( ivar, histo, varInfo, outputDir, outputLabel='data', addCorrelation=False, addCondition=False, addInvertedMatrix=False,ext='pdf' ):
    """docstring for draw2D"""

    if not os.path.exists(outputDir): os.makedirs(outputDir)
    outputName = outputDir+ivar+'_'+selection+'_'+outputLabel+'_'+version+'.'+ext


    ROOT.gStyle.SetPadRightMargin(0.15)
    can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750, 500 )
    histo.GetXaxis().SetTitle('Accepted Gen '+varInfo['label'])
    histo.GetYaxis().SetTitle('True Reco '+varInfo['label'])
    histo.GetYaxis().SetTitleOffset( 0.8 )
    #signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel]covHisto.GetYaxis().SetRange( variables[ivar]['bins'][0], variables[ivar]['bins'][-1] )
    histo.Draw("colz")
    CMS_lumi.extraText = "Simulation Preliminary"
    CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
    CMS_lumi.relPosX = 0.12
    CMS_lumi.CMS_lumi(can2D, 4, 0)

    textBox.SetTextSize(0.04)
    if addCorrelation:
        print('|-----> Correlation: ', histo.GetCorrelationFactor())
        textBoxCorr = textBox.Clone()
        textBoxCorr.DrawLatex( 0.05, varInfo['bins'][-1]-( .05*(varInfo['bins'][-1]-varInfo['bins'][0]) ), '#color[8]{Corr. Factor = '+str(round(histo.GetCorrelationFactor(),2))+'}' )

    if addCondition:   ## based on https://gitlab.cern.ch/DasAnalysisSystem/InclusiveJet/-/blob/master/UnfoldingSampleND/bin/unfold.cc#L41
        Nx = histo.GetNbinsX()
        Ny = histo.GetNbinsY()
        RMx = histo.ProjectionX( 'RMx', 0, -1 )

        m = ROOT.TMatrixD( Ny, Nx )   ### need to swap the axes
        for ibin in range(1, Nx+1):
            normalization = RMx.GetBinContent(ibin)
            if (normalization>0):
                for jbin in range( 1, Ny+1 ):
                    m[jbin-1][ibin-1] = histo.GetBinContent(ibin,jbin) / normalization
        svd = ROOT.TDecompSVD(m)
        v = ROOT.TVectorD( svd.GetSig() )
        Min = v[0]
        Max = v[0]
        for ibin in range( 0, Nx ):
            if (abs(v[ibin]) < 1e-5 ): break
            Min = v[ibin]
        conditionNumber = str(round( Max/Min, 2 )) if Min > 0 else 'NaN'
        print('|-----> Condition Number: ', conditionNumber)
        textBoxCond = textBox.Clone()
        textBoxCond.DrawLatex( 0.05, varInfo['bins'][-1]-( .1*(varInfo['bins'][-1]-varInfo['bins'][0]) ), '#color[8]{Cond. Number = '+conditionNumber+'}' )

    can2D.SaveAs(outputName)
    if ext.startswith('pdf'):
        can2D.SaveAs( outputName.replace('pdf', 'png') )

    ROOT.gStyle.SetPadRightMargin(0.09)     ## reseating
    ROOT.gStyle.SetPadLeftMargin(0.12)



In [14]:
def Scale(h):
    n=h.Integral()
    h.Scale(1./n,"width")

In [15]:
def getFiles(labelBegin='', year_list=['2017','2018','all']):
    Files={}
    for iy in year_list:
        for isam in dictSamples:
            if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
            if isam.startswith(labelBegin):
                Files[isam] = [
                                ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                                checkDict( isam, dictSamples )
                            ]
    return Files

In [29]:
colors = [ 2, 4,  9, 8, 28, 30, 42, 13, 12, 40, 46, 3, 24, 26, 41, 45, 48, 49, 37, 38, 33, 17]
##########################################################################
def runTUnfold( dataFile, sigFiles, bkgFiles, variables, sel, sysUncert, process, ext, runMLU=False, sysSygFiles=[]):

    """docstring for createDataCards"""

    for ivar in variables:

        outputDir=outputFolder+sel.split('_')[1]+'/'+year+'/Unfolding/'+ivar+'/'+process+'/'
        if not os.path.exists(outputDir): os.makedirs(outputDir)

        if year.startswith('all'):
            signalHistos = {
                    signalLabel+'_resp'+ivar+'_nom'+sel : dataFile[ivar+'_2017'].Get(signalLabel+'_resp'+ivar+'_nom'+sel),
                    signalLabel+'_reco'+ivar+'_nom'+sel : dataFile[ivar+'_2017'].Get(signalLabel+'_reco'+ivar+'_nom'+sel),
                    signalLabel+'_fakereco'+ivar+'_nom'+sel : dataFile[ivar+'_2017'].Get(signalLabel+'_fakereco'+ivar+'_nom'+sel),
                    signalLabel+'_accepgen'+ivar+sel : dataFile[ivar+'_2017'].Get(signalLabel+'_accepgen'+ivar+sel),
                    signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin': dataFile[ivar+'_2017'].Get(signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin')
                    }
            if not process.startswith('MCCrossClosure'):
                signalHistos[ signalLabel+'_resp'+ivar+'_nom'+sel ].Add( dataFile[ivar+'_2018'].Get(signalLabel+'_resp'+ivar+'_nom'+sel) )
                signalHistos[ signalLabel+'_reco'+ivar+'_nom'+sel ].Add( dataFile[ivar+'_2018'].Get(signalLabel+'_reco'+ivar+'_nom'+sel) )
                signalHistos[ signalLabel+'_fakereco'+ivar+'_nom'+sel ].Add( dataFile[ivar+'_2018'].Get(signalLabel+'_fakereco'+ivar+'_nom'+sel) )
                signalHistos[ signalLabel+'_accepgen'+ivar+sel ].Add( dataFile[ivar+'_2018'].Get(signalLabel+'_accepgen'+ivar+sel) )
                signalHistos[ signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin' ].Add( dataFile[ivar+'_2018'].Get(signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin') )
            
            sysSignalHistos={}
            for sys in sysUncert:
                for upDown in [ 'Up', 'Down' ]:
                    if sys.startswith('_model'): continue
                    s = [i for i in sysSignalLabels if sys in i]
                    print (s[0]+'_reco'+ivar+sys+upDown+sel)
                    sysSignalHistos[ s[0]+'_reco'+ivar+sys+upDown+sel ] = dataFile[ivar+'_2017'].Get(s[0]+'_reco'+ivar+sys+upDown+sel)
                    if not process.startswith('MCCrossClosure'): sysSignalHistos[ s[0]+'_reco'+ivar+sys+upDown+sel ].Add( dataFile[ivar+'_2018'].Get(s[0]+'_reco'+ivar+sys+upDown+sel) )
                    sysSignalHistos[ s[0]+'_resp'+ivar+sys+upDown+sel ] = dataFile[ivar+'_2017'].Get(s[0]+'_resp'+ivar+sys+upDown+sel)
                    if not process.startswith('MCCrossClosure'): sysSignalHistos[ s[0]+'_resp'+ivar+sys+upDown+sel ].Add( dataFile[ivar+'_2018'].Get(s[0]+'_resp'+ivar+sys+upDown+sel) )

            altSignalHistos = {
                altSignalLabel+'_resp'+ivar+'_nom'+sel : dataFile[ivar+'_2017'].Get(altSignalLabel+'_resp'+ivar+'_nom'+sel),
                altSignalLabel+'_reco'+ivar+'_nom'+sel : dataFile[ivar+'_2017'].Get(altSignalLabel+'_reco'+ivar+'_nom'+sel),
                altSignalLabel+'_fakereco'+ivar+'_nom'+sel : dataFile[ivar+'_2017'].Get(altSignalLabel+'_fakereco'+ivar+'_nom'+sel),
                altSignalLabel+'_accepgen'+ivar+sel : dataFile[ivar+'_2017'].Get(altSignalLabel+'_accepgen'+ivar+sel),
                }

            if not process.startswith('MCCrossClosure'):
                altSignalHistos[ altSignalLabel+'_resp'+ivar+'_nom'+sel ].Add( dataFile[ivar+'_2018'].Get(altSignalLabel+'_resp'+ivar+'_nom'+sel) )
                altSignalHistos[ altSignalLabel+'_reco'+ivar+'_nom'+sel ].Add( dataFile[ivar+'_2018'].Get(altSignalLabel+'_reco'+ivar+'_nom'+sel) )
                altSignalHistos[ altSignalLabel+'_fakereco'+ivar+'_nom'+sel ].Add( dataFile[ivar+'_2018'].Get(altSignalLabel+'_fakereco'+ivar+'_nom'+sel) )
                altSignalHistos[ altSignalLabel+'_accepgen'+ivar+sel].Add( dataFile[ivar+'_2018'].Get(altSignalLabel+'_accepgen'+ivar+sel) )

            allHistos = {
                    'dataHisto' : dataFile[ivar+'_2017'].Get( 'dataHisto' ),
                    'dataHistoGenBin' : dataFile[ivar+'_2017'].Get( 'dataHistoGenBin' )
                    }
            allHistos[ 'dataHisto' ].Add( dataFile[ivar+'_2018'].Get( 'dataHisto' ) )
            allHistos[ 'dataHistoGenBin' ].Add( dataFile[ivar+'_2018'].Get( 'dataHistoGenBin' ) )

            if process.startswith('MCCrossClosure'):
                allHistos[ 'dataHisto' ] = dataFile[ivar+'_2018'].Get(signalLabel+'_reco'+ivar+'_nom'+sel)
                allHistos[ 'dataHistoGenBin' ] = dataFile[ivar+'_2018'].Get(signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin')
            
            #sysSignalHistos = loadHistograms( sysSigFiles, ivar, sel, sysUnc=[ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal'], respOnly=False, isMC=True, lumi=lumi, year=year, process=process, variables=variables)


            dataHistos = { }
            bkgHistos = { }

        else:
            print('|-------> Running single year '+year)
            ### Getting input histos
            mainSigFiles = { k:v for (k,v) in sigFiles.items() if k.startswith(signalLabelBegin)  }
            signalHistos = loadHistograms( mainSigFiles, ivar, sel, sysUnc=[], respOnly=False, lumi=lumi, year=year, process=process, variables=variables )
            tmp2SigFiles = { k:v for (k,v) in sigFiles.items() if k.startswith(altSignalLabelBegin)  }
            altSignalHistos = loadHistograms( tmp2SigFiles, ivar, sel, sysUnc=[], isMC=True, respOnly=False, lumi=lumi, year=year, process=process, variables=variables)
            sysSignalHistos = loadHistograms( sysSigFiles, ivar, sel, sysUnc=[ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal'], respOnly=False, isMC=True, lumi=lumi, year=year, process=process, variables=variables)
            #print ("Alt histos:", altSignalHistos)
            if process.startswith("MC"):
                if process.startswith('MCSelfClosure'):
                    dataHistos = { 'data_reco'+k.split(('_reco' if sel.startswith('_dijet') else 'Leptonic'))[1] : v for (k,v) in signalHistos.items() if '_reco' in k }
                else:
                    dataHistos = loadHistograms( tmp2SigFiles, ivar, sel, isMC=True, addGenInfo=False, lumi=lumi, year=year, process=process, variables=variables )
                    dataHistos = { 'data_reco'+k.split('_reco')[1] : v for (k,v) in dataHistos.items() if '_reco' in k }
                    #print(dataHistos)
            else:
                dataHistos = loadHistograms( dataFile, ivar, sel, isMC= False, lumi=lumi, year=year, process=process, variables=variables )
            bkgHistos = loadHistograms( bkgFiles, ivar, sel, sysUnc=[], lumi=lumi, year=year, process=process, variables=variables) if process.startswith('data') else {}

            ####### Fix fake and true reco
            signalHistos[signalLabel+'_truereco'+ivar+'_nom'+sel]= signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].ProjectionY()
            signalHistos[signalLabel+'_fakereco'+ivar+'_nom'+sel] = signalHistos[signalLabel+'_reco'+ivar+'_nom'+sel].Clone()
            signalHistos[signalLabel+'_fakereco'+ivar+'_nom'+sel].Add( signalHistos[signalLabel+'_truereco'+ivar+'_nom'+sel], -1 )

            signalHistos[signalLabel+'_accepgen'+ivar+sel] = signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].ProjectionX()
            signalHistos[signalLabel+'_missgen'+ivar+sel] = signalHistos[signalLabel+'_gen'+ivar+sel].Clone()
            signalHistos[signalLabel+'_missgen'+ivar+sel].Add( signalHistos[signalLabel+'_accepgen'+ivar+sel], -1 )
            
            altSignalHistos[altSignalLabel+'_truereco'+ivar+'_nom'+sel]= altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].ProjectionY()
            altSignalHistos[altSignalLabel+'_fakereco'+ivar+'_nom'+sel] = altSignalHistos[altSignalLabel+'_reco'+ivar+'_nom'+sel].Clone()
            altSignalHistos[altSignalLabel+'_fakereco'+ivar+'_nom'+sel].Add( altSignalHistos[altSignalLabel+'_truereco'+ivar+'_nom'+sel], -1 )

            altSignalHistos[altSignalLabel+'_accepgen'+ivar+sel] = altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].ProjectionX()
            altSignalHistos[altSignalLabel+'_missgen'+ivar+sel] = altSignalHistos[altSignalLabel+'_gen'+ivar+sel].Clone()
            altSignalHistos[altSignalLabel+'_missgen'+ivar+sel].Add( altSignalHistos[altSignalLabel+'_accepgen'+ivar+sel], -1 )

            print ('|------> Unfolding '+ivar)

            ######## Cross check: plotting data vs all MC Scaled
            print ('|------> Cross check: plotting data vs all MC')
            allHistos = {}
            allHistos[ 'allBkgHisto' ] = dataHistos['data_reco'+ivar+'_nom'+sel].Clone()
            allHistos[ 'allBkgHisto' ].Reset()
            allHistos[ 'allBkgHistoGenBin' ] = dataHistos['data_reco'+ivar+'_nom'+sel+'_genBin'].Clone()
            allHistos[ 'allBkgHistoGenBin' ].Reset()
            for ibkg in bkgHistos:
                if ibkg.endswith('_reco'+ivar+'_nom'+sel): allHistos[ 'allBkgHisto' ].Add( bkgHistos[ibkg].Clone() )
                if ibkg.endswith('_reco'+ivar+'_nom'+sel+'_genBin'): allHistos[ 'allBkgHistoGenBin' ].Add( bkgHistos[ibkg].Clone() )
            allHistos[ 'allMCHisto' ] = allHistos[ 'allBkgHisto' ].Clone()
            allHistos[ 'allMCHisto' ].Add( signalHistos[ signalLabel+'_reco'+ivar+'_nom'+sel ].Clone() )
            allHistos[ 'allMCHistoGenBin' ] = allHistos[ 'allBkgHistoGenBin' ].Clone()
            allHistos[ 'allMCHistoGenBin' ].Add( signalHistos[ signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin' ].Clone() )
            ### For dijet, scale QCD to data
            scaleFactor =1
            if sel.startswith('_dijet'):
                scaleFactor = dataHistos['data_reco'+ivar+'_nom'+sel].Integral() / allHistos[ 'allMCHisto' ].Integral()
                scaleFactorGenBin = dataHistos['data_reco'+ivar+'_nom'+sel+'_genBin'].Integral() / allHistos[ 'allMCHistoGenBin' ].Integral()
                allHistos[ 'allMCHisto' ].Scale( scaleFactor )
                allHistos[ 'allBkgHisto' ].Scale( scaleFactor )
                allHistos[ 'allBkgHistoGenBin' ].Scale( scaleFactorGenBin )
                for ihsig in signalHistos:
                    if ihsig.endswith(sel):
                        #print (ihsig)
                        signalHistos[ihsig].Scale( scaleFactor )
                    if ihsig.endswith('genBin'): signalHistos[ihsig].Scale( scaleFactorGenBin )
                    #if 'resp' in ihsig: signalHistos[ihsig].Scale( scaleFactor )   ### dont notice the difference

            plotSimpleComparison( dataHistos['data_reco'+ivar+'_nom'+sel].Clone(), 'data', allHistos[ 'allMCHisto' ], 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+signalLabel+"_nom", rebinX=variables[ivar]['bins'], version=sel+'_'+version, outputDir=outputDir )

#            ######## Adding missgen to overflow and underflow
#            for ibin in range( 1, signalHistos[signalLabel+'_missgen'+ivar+sel].GetNbinsX()+1 ):
#                signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].SetBinContent(  signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetBin( 0, ibin ), signalHistos[signalLabel+'_missgen'+ivar+sel].GetBinContent(ibin) )
#                signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].SetBinError(  signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetBin( 0, ibin ), signalHistos[signalLabel+'_missgen'+ivar+sel].GetBinError(ibin) )
#                signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].SetBinContent(  signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetBin( -1, ibin ), signalHistos[signalLabel+'_missgen'+ivar+sel].GetBinContent(ibin) )
#                signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].SetBinError(  signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetBin( -1, ibin ), signalHistos[signalLabel+'_missgen'+ivar+sel].GetBinError(ibin) )
#                altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].SetBinContent(  altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].GetBin( 0, ibin ), altSignalHistos[altSignalLabel+'_missgen'+ivar+sel].GetBinContent(ibin) )
#                altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].SetBinError(  altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].GetBin( 0, ibin ), altSignalHistos[altSignalLabel+'_missgen'+ivar+sel].GetBinError(ibin) )
#                altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].SetBinContent(  altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].GetBin( -1, ibin ), altSignalHistos[altSignalLabel+'_missgen'+ivar+sel].GetBinContent(ibin) )
#                altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].SetBinError(  altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].GetBin( -1, ibin ), altSignalHistos[altSignalLabel+'_missgen'+ivar+sel].GetBinError(ibin) )
##                #for sys in sysUncert:
##                #    for upDown in [ 'Up', 'Down' ]:
##                #        signalHistos[signalLabel+'_resp'+ivar+sys+upDown+sel].SetBinContent(  signalHistos[signalLabel+'_resp'+ivar+sys+upDown+sel].GetBin( ibin, 0 ), signalHistos[signalLabel+'_missgen'+ivar+sel].GetBinContent(ibin) )
##                #        signalHistos[signalLabel+'_resp'+ivar+sys+upDown+sel].SetBinContent(  signalHistos[signalLabel+'_resp'+ivar+sys+upDown+sel].GetBin( ibin, -1 ), signalHistos[signalLabel+'_missgen'+ivar+sel].GetBinContent(ibin) )

            ####### Cross check response matrix
            tmpGenHisto = signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].ProjectionX()
            plotSimpleComparison( tmpGenHisto, 'projection', signalHistos[signalLabel+'_accepgen'+ivar+sel].Clone(), 'Regular AccepGen', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+signalLabel+"_TestProjectionGen", rebinX=1, version=sel+'_'+version, outputDir=outputDir )
            tmpRecoHisto = signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].ProjectionY()
            plotSimpleComparison( tmpRecoHisto, 'projection', signalHistos[signalLabel+'_truereco'+ivar+'_nom'+sel].Clone(), 'Regular TrueReco', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+signalLabel+"_TestProjectionReco", rebinX=1, version=sel+'_'+version, outputDir=outputDir )
            tmpRecoHisto.Scale( scaleFactor )
            
            ####### plotting Removing of bkgs from data
            fakeHistos = { k:v for (k,v) in signalHistos.items() if 'fakereco' in k }
            for ih in fakeHistos:
                if ih.endswith(ivar+'_nom'+sel): allHistos[ 'allBkgHisto' ].Add( fakeHistos[ih] )
                if ih.endswith(ivar+'_nom'+sel+'_genBin'): allHistos[ 'allBkgHistoGenBin' ].Add( fakeHistos[ih] )
            plotSimpleComparison( dataHistos[ 'data_reco'+ivar+'_nom'+sel ].Clone(), 'data', allHistos[ 'allBkgHisto' ], 'Bkg+fakes', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+signalLabel+"_TestDataBkgFakes", rebinX=1, version=sel+'_'+version, outputDir=outputDir )

            allHistos[ 'dataHisto' ] = dataHistos[ 'data_reco'+ivar+'_nom'+sel ].Clone()
            #allHistos[ 'dataHisto' ].Add( allHistos[ 'allBkgHisto' ].Clone(), -1 )
            allHistos[ 'dataHisto' ].Scale( 1/allHistos[ 'dataHisto' ].Integral() )
            allHistos[ 'dataHistoGenBin' ] = dataHistos[ 'data_reco'+ivar+'_nom'+sel+'_genBin' ].Clone()
            #allHistos[ 'dataHistoGenBin' ].Add( allHistos[ 'allBkgHistoGenBin' ].Clone(), -1 )
            allHistos[ 'dataHistoGenBin' ].Scale( 1/allHistos[ 'dataHistoGenBin' ].Integral() )

            allHistos[ 'dataMinusBkgs' ] = dataHistos[ 'data_reco'+ivar+'_nom'+sel ].Clone()
            allHistos[ 'dataMinusBkgs' ].Add( allHistos[ 'allBkgHisto' ].Clone(), -1 )
            allHistos[ 'dataMinusBkgs' ].Scale( 1/allHistos[ 'dataMinusBkgs' ].Integral() )
            allHistos[ 'dataMinusBkgsGenBin' ] = dataHistos[ 'data_reco'+ivar+'_nom'+sel+'_genBin' ].Clone()
            allHistos[ 'dataMinusBkgsGenBin' ].Add( allHistos[ 'allBkgHistoGenBin' ].Clone(), -1 )
            allHistos[ 'dataMinusBkgsGenBin' ].Scale( 1/allHistos[ 'dataMinusBkgsGenBin' ].Integral() )

            tmpHisto = signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].ProjectionY()
            tmpHisto.Scale( 1/tmpHisto.Integral() )

            tmpHisto = signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].ProjectionY()
            tmpHisto.Scale( 1/tmpHisto.Integral() )
            plotSimpleComparison( allHistos[ 'dataMinusBkgs' ], 'data-Bkgs', tmpHisto, 'signal true reco', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+signalLabel+"_TestDataMinusBkgs", rebinX=1, version=sel+'_'+version, outputDir=outputDir )


            
        ######## Cross check: plotting response matrix
        print ('|------> Cross check: plotting response matrix for signal')
        ROOT.gStyle.SetPadRightMargin(0.15)
        #ROOT.gStyle.SetPalette(ROOT.kGistEarth)
        #ROOT.TColor.InvertPalette()
        can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750, 500 )
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetXaxis().SetTitle('Accepted Gen '+variables[ivar]['label'])
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetYaxis().SetTitle('True Reco '+variables[ivar]['label'])
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetYaxis().SetTitleOffset( 0.8 )
        #signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetYaxis().SetRange( variables[ivar]['bins'][0], variables[ivar]['bins'][-1] )
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].Draw("colz")
        CMS_lumi.extraText = "Simulation Preliminary"
        CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
        CMS_lumi.relPosX = 0.12
        CMS_lumi.CMS_lumi(can2D, 4, 0)
        can2D.SaveAs(outputDir+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+signalLabel+sel+'_responseMatrix'+version+'.'+ext)

        

        ######## TUnfold part
        print ('|------> TUnfolding starts:')

        ##### Defining options for TUnfold
        tunfolder = ROOT.TUnfoldDensity(
                                            signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel], ### response matrix. According to TUnfold, this distribution does NOT have to be normalized
                                            ROOT.TUnfold.kHistMapOutputHoriz,  #### kHistMapOutputVert if x->reco and y->gen, kHistMapOutputHoriz if x->gen and y->reco
                                            ROOT.TUnfold.kRegModeCurvature,   ##### Regularization Mode : ROOT.TUnfold.kRegModeCurvature regularizes based on the 2nd derivative of the output. More information wrt the other options can be gained from reading the source code
                                            ROOT.TUnfold.kEConstraintNone,    ##### Constraint : TUnfold.kEConstraintNone meaning we do not constrain further, the other option is to force constraint of area. (Need to look into this!!)
                                            ROOT.TUnfoldDensity.kDensityModeBinWidth  ##### Density Mode: ROOT.TUnfoldDensity.kDensityModeBinWidth uses the bin width to normalize the event rate in a given bin, accounting for non-uniformity in bin widths as discussed in section 7.2.1 of the TUnfold paper
                                            )

        ##### Defining input (data recoJet )
        print ('|------> TUnfolding adding input:')
        #tunfolder.SetInput( dataHistos[ 'data_reco'+ivar+'_nom'+sel ].Clone() )
        print (allHistos.keys())
        tunfolder.SetInput( allHistos[ 'dataHisto' ])
        print ("Subtracting backgrounds")
        dummy=0
        
        if sel.startswith(('_W','_top')):
            bkgSources = []

            for ibkg in bkgHistos:
                if ibkg.endswith('_reco'+ivar+'_nom'+sel):
                    print (ibkg.split('_')[0]+ '%d'%dummy)
                    tunfolder.SubtractBackground( bkgHistos[ibkg].Clone(), ibkg.split('_')[0]+ '%d'%dummy )
                    dummy=dummy+1
                    bkgSources.append(ibkg.split('_')[0]+ '%d'%dummy)
                if ibkg.endswith('_reco'+ivar+'_nom'+sel+'_genBin'): allHistos[ 'allBkgHistoGenBin' ].Add( bkgHistos[ibkg].Clone() )

        tunfolder.SubtractBackground( signalHistos[signalLabel+'_fakereco'+ivar+'_nom'+sel], 'fakes')
        

        ###### Adding SYS unc
        if len(sysUncert)>0 :
            print ('|------> TUnfolding adding uncert:')
            dictUncHistos = {}
            for sys in sysUncert:
                #print (sys)
                if sys in ([ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal' ]):
                    print('|------> TUnfolding adding %sUnc'%sys)
                    s = [i for i in sysSignalLabels if sys in i]
                    #print (s)
                    dictUncHistos[sys+' Up'] = sysSignalHistos[s[0]+'_reco'+ivar+sys+'Up'+sel].Clone()
                    dictUncHistos[sys+' Down'] = sysSignalHistos[s[0]+'_reco'+ivar+sys+'Down'+sel].Clone()
                    for upDown in [ 'Up', 'Down' ]:
                        #print (sys+upDown)
                        tunfolder.AddSysError(
                                            sysSignalHistos[s[0]+'_resp'+ivar+sys+upDown+sel],
                                            sys+upDown,
                                            ROOT.TUnfold.kHistMapOutputHoriz,
                                            ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                            )
                        can2DNorm = ROOT.TCanvas(ivar+'can2DNorm'+sys+upDown, ivar+'can2DNorm'+sys+upDown, 750, 500 )
                        sysSignalHistos[s[0]+'_reco'+ivar+sys+upDown+sel].Draw("colz")
                        can2DNorm.SaveAs(outputDir+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+s[0]+sel+upDown+'Normalized_responseMatrix'+version+'.'+ext)

                #### adding model uncertainty
                elif sys.startswith(('_model')):
                    #if not process.startswith('MCSelfClosure'):
                    print('|------> TUnfolding adding modelUnc')
                    tunfolder.AddSysError(
                                        altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel],
                                        'modelUncTotal',
                                        ROOT.TUnfold.kHistMapOutputHoriz,
                                        ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                        )
                    can2DNorm = ROOT.TCanvas(ivar+'can2DNormAltSignal', ivar+'can2DNormAltSignal', 750, 500 )
                    altSignalHistos[altSignalLabel+'_resp'+ivar+'_nom'+sel].Draw("colz")
                    can2DNorm.SaveAs(outputDir+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+altSignalLabel+sel+'Normalized_alt_responseMatrix'+version+'.'+ext)
                    dictUncHistos[sys] = altSignalHistos[altSignalLabel+'_reco'+ivar+'_nom'+sel].Clone()
            #### Making unc plot
            plotSysComparison( signalHistos[signalLabel+'_reco'+ivar+'_nom'+sel].Clone(),
                                dictUncHistos,
                                ivar+'_'+signalLabel+'_allSys',
                                labelX=variables[ivar]['label'],
                                version=sel+'_'+version,
                                year= ( '2017+2018' if year.startswith('all') else year ),
                                outputDir=outputDir
                                )

        ###### Running the unfolding
        print ('|------> TUnfolding doUnfold:')
        tunfolder.DoUnfold(0)

        ##### Get output of unfolding
        allHistos [ 'unfoldHisto'+ivar ] = tunfolder.GetOutput("unfoldHisto"+ivar).Clone()
        unfoldingtot = allHistos [ 'unfoldHisto'+ivar ].Integral()

        allHistos [ 'foldHisto'+ivar ] = tunfolder.GetFoldedOutput("folded"+ivar).Clone()

        #### Get Probability matrix
        allHistos[ 'probaMatrix'+ivar ] = tunfolder.GetProbabilityMatrix('probaMatrix'+ivar).Clone()
        
        
        ###################################################### Get systematic shifts of output #######################################################
        uncerUnfoldHisto = OrderedDict() #storing individual systematics totals (ie, up/down or other variations), background subtraction systematics, and then the overall systematic unc (individual systs + bkgs)

        if len(sysUncert)>0 :
            print ('|------> TUnfolding uncertainties:')

            for sys in sysUncert:
                if not sys.startswith('_model'):
                    for upDown in [ 'Up', 'Down' ]:
                        #print (sys+upDown)
                        uncerUnfoldHisto[ivar+sys+upDown] = tunfolder.GetDeltaSysSource(sys+upDown, "unfoldHisto_"+ivar+sys+upDown+"shift", "-1#sigma")
                        try: uncerUnfoldHisto[ivar+sys+upDown].SetLineStyle(1)
                        except ReferenceError: uncerUnfoldHisto.pop( ivar+sys+upDown, None )

                    # Create total uncertainty and sys uncertainty plots.
                    uncerUnfoldHisto[ivar+sys.upper()+'Total'] = allHistos[ 'unfoldHisto'+ivar ].Clone()       # Syst uncertainty
                    uncerUnfoldHisto[ivar+sys.upper()+'Total'].Reset()
                    for i in range( 0, allHistos[ 'unfoldHisto'+ivar ].GetNbinsX() + 1):
                        try: yup = abs( uncerUnfoldHisto[ivar+sys+'Up'].GetBinContent(i) )
                        except KeyError: yup = 0
                        try: ydn = abs( uncerUnfoldHisto[ivar+sys+'Down'].GetBinContent(i) )
                        except KeyError: ydn = 0
                        dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
                        uncerUnfoldHisto[ivar+sys.upper()+'Total'].SetBinContent( i, dy )

                else:
                    #if not process.startswith('MCSelfClosure'):
                    uncerUnfoldHisto[ivar+'_Physics ModelTotal'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_modelUncTotal')
                    uncerUnfoldHisto[ivar+'_Physics ModelTotal'].Reset()
                    tmpModelHisto = tunfolder.GetDeltaSysSource('modelUncTotal', "unfoldHisto_"+ivar+"modelUncshift", "-1#sigma")
                    for i in range( 0, tmpModelHisto.GetNbinsX() + 1):
                        uncerUnfoldHisto[ivar+'_Physics ModelTotal'].SetBinContent( i, abs(tmpModelHisto.GetBinContent(i)) )
                    #print('_modelUncTotal')

        #####################################################################################################################################################

        
        
        
        
        
        ################################################################################  Get various covariances  ###########################################################################################################
        '''
        GetEmatrixSysUncorr(): uncorrelated errors on the input matrix histA, taken as the errors provided with the histogram. These are typically statistical errors from finite Monte Carlo samples.
        GetEmatrixSysSource()(GetDeltaSysSource()): correlated shifts of the input matrix histA. These shifts are taken as one-sigma effects when switchig on a given error soure. Several such error sources may be defined
        GetEmatrixSysBackgroundUncorr(): uncorrelated errors on background sources, originating from the errors provided with the background histograms
        GetEmatrixInput(): statistical uncertainty of the input (the measurement)
        
        '''
        
        print ('|------> TUnfolding covariances')
        allHistos[ 'cov'+ivar ] = tunfolder.GetEmatrixTotal("cov"+ivar, "Covariance Matrix")
        allHistos[ 'cov_uncorr_data_'+ivar ] = tunfolder.GetEmatrixInput("cov_uncorr_data"+ivar, "Covariance Matrix coming from input data distributions")
        allHistos[ 'cov_uncorr_'+ivar ] = tunfolder.GetEmatrixSysUncorr("cov_uncorr"+ivar, "Covariance Matrix at hadron level coming from RM")
        allHistos[ 'cov_uncorr_bkg_'+ivar ] = tunfolder.GetEmatrixSysBackgroundUncorr('fakes', "Covariance Matrix from uncorrelated errors of Background sources")
        if sel.startswith(('_W','_top')):
            for ibkg in bkgSources[1:]:
                allHistos[ 'cov_uncorr_bkg_'+ivar ].Add(tunfolder.GetEmatrixSysBackgroundUncorr(ibkg, "Covariance Matrix from uncorrelated errors of Background sources"))
        #### cov = cov_uncorr + cov_uncorr_data + other uncertaitnies
        #### stat = cov_uncorr + cov_uncorr_data
        
        
        ############### Build correlation matrix for unfolding#################################
        allHistos['correlation_matrix_'+ivar] = allHistos[ 'cov'+ivar ].Clone()
        allHistos['correlation_matrix_'+ivar].Reset()
        
        for ibinx in range( 1, allHistos[ 'cov'+ivar ].GetNbinsX()):
            variance = allHistos[ 'cov'+ivar ].GetBinContent(ibinx,ibinx)

            for ibiny in range( 1, allHistos[ 'cov'+ivar ].GetNbinsY()):
                cov_xy = allHistos[ 'cov'+ivar ].GetBinContent(ibinx,ibiny)
                if variance!=0: 
                    allHistos['correlation_matrix_'+ivar].SetBinContent(ibinx,ibiny,cov_xy/variance)
        ########################################################################################    
        
        # Create total uncertainty and sys uncertainty histos for plots 
        
        # first build up histos of systematic uncertainties from the BACKGROUND SUBTRACTION 
        uncerUnfoldHisto[ivar+'_Bkg.'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_Bkg. SystTotal')
        uncerUnfoldHisto[ivar+'_Bkg.'].Reset()
        for ibin in range( 0, allHistos[ 'unfoldHisto'+ivar ].GetNbinsX()+1 ):
            bkg_tot = ROOT.TMath.Sqrt(allHistos[ 'cov_uncorr_bkg_'+ivar ].GetBinContent(ibin,ibin))
            uncerUnfoldHisto[ivar+'_Bkg.'].SetBinContent(ibin, bkg_tot)
            
        # second build up histos of total systematic uncertainties
        uncerUnfoldHisto[ivar+'_SystTotal'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_SystTotal')
        uncerUnfoldHisto[ivar+'_SystTotal'].Reset()        
        
        tmp = OrderedDict()
        for i in range( 0, allHistos[ 'unfoldHisto'+ivar ].GetNbinsX() + 1):
            tmp[i] = 0
            for k in uncerUnfoldHisto:
                if k.endswith('Total') and not k.endswith(('SystTotal')):
                    tmp[i] = tmp[i] + ( uncerUnfoldHisto[k].GetBinContent( i )**2 )
                    #print(i, k, tmp[i], uncerUnfoldHisto[k].GetBinContent( i ), ( uncerUnfoldHisto[k].GetBinContent( i )**2 ))
        
        if len(sysUncert)>0: 
            for i,j in tmp.items():
                uncerUnfoldHisto[ivar+'_SystTotal'].SetBinContent( i, ROOT.TMath.Sqrt( j ) )
        
        uncerUnfoldHisto[ivar+'_StatTotal'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_StatTotal')
        uncerUnfoldHisto[ivar+'_StatTotal'].Reset()
        uncerUnfoldHisto[ivar+'_TotalUnc'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_TotalUnc')
        uncerUnfoldHisto[ivar+'_TotalUnc'].Reset()
        
        uncerUnfoldHisto[ivar+'_CMTotal'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_CMTotal')
        uncerUnfoldHisto[ivar+'_CMTotal'].Reset()
        uncerUnfoldHisto[ivar+'_CMUncorrTotal'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_CMUncorrTotal')
        uncerUnfoldHisto[ivar+'_CMUncorrTotal'].Reset()
        uncerUnfoldHisto[ivar+'_CMInpStatTotal'] = allHistos[ 'unfoldHisto'+ivar ].Clone(ivar+'_CMInpStatTotal')
        uncerUnfoldHisto[ivar+'_CMInpStatTotal'].Reset()
        
        allHistos[ 'unfoldHistowoUnc'+ivar ] = allHistos[ 'unfoldHisto'+ivar ].Clone()        # No unc
        allHistos[ 'unfoldHistoStatUnc'+ivar ] = allHistos[ 'unfoldHisto'+ivar ].Clone("unfoldHistoStatUnc")     # Unfolding and stat unc
        allHistos[ 'unfoldHistoBkgUnc'+ivar ] = allHistos[ 'unfoldHisto'+ivar ].Clone("unfoldHistoBkgStatUnc")   # Bkg subtraction unc.
        allHistos[ 'unfoldHistoSystUnc'+ivar ] = allHistos[ 'unfoldHisto'+ivar ].Clone("unfoldHistoSystUnc")     # Unc. from systematics and variations
        #allHistos[ 'unfoldHistoTotUnc'+ivar ] = allHistos[ 'unfoldHisto'+ivar ].Clone("unfoldHistoTotUnc")       # Total uncertainty

        
        ratioHistos = OrderedDict()
        ratioHistos[ 'StatUnc'+ivar ] = uncerUnfoldHisto[ivar+'_StatTotal'].Clone('StatUnc'+ivar)   
        ratioHistos[ 'StatUnc'+ivar ].Reset()
        ratioHistos[ 'TotalUnc'+ivar ] = uncerUnfoldHisto[ivar+'_StatTotal'].Clone('TotalUnc'+ivar)   
        ratioHistos[ 'TotalUnc'+ivar ].Reset()
        ratioHistos[ 'SystUnc'+ivar ] = uncerUnfoldHisto[ivar+'_StatTotal'].Clone('SystUnc'+ivar)
        ratioHistos[ 'SystUnc'+ivar ].Reset()
        
        print ("BC of unfHist", "systot+cov in quadrature", "tot from cov", "syst tot", 'stat+unf unc from unfhisto')
        
        for ibin in range( 0, allHistos[ 'unfoldHisto'+ivar ].GetNbinsX()+1 ):
            
            unc_tot = ROOT.TMath.Sqrt( allHistos[ 'cov'+ivar ].GetBinContent(ibin,ibin) ) #total from covariances, so not including systematics
            bkg_tot = ROOT.TMath.Sqrt(allHistos[ 'cov_uncorr_bkg_'+ivar ].GetBinContent(ibin,ibin))
            if len(sysUncert)>0: syst_tot = uncerUnfoldHisto[ivar+'_SystTotal'].GetBinContent(ibin)
            else: syst_tot=0
            orig_statunfunc = allHistos[ 'unfoldHisto'+ivar ].GetBinError(ibin) #obtain the stat + unf uncertatinties directly from output (which by default contains no systematics)
            
            norm_unc = abs(allHistos[ 'unfoldHisto'+ivar ].GetBinContent(ibin))
            
            allHistos[ 'unfoldHisto'+ivar ].SetBinError(ibin, unc_tot)
            allHistos[ 'unfoldHistoBkgUnc'+ivar ].SetBinError(ibin, bkg_tot )

            uncerUnfoldHisto[ivar+'_TotalUnc'].SetBinContent(ibin, ROOT.TMath.Sqrt(unc_tot**2+syst_tot**2 ))
            uncerUnfoldHisto[ivar+'_CMTotal'].SetBinContent(ibin, ROOT.TMath.Sqrt(unc_tot**2))
            uncerUnfoldHisto[ivar+'_CMUncorrTotal'].SetBinContent(ibin, ROOT.TMath.Sqrt( allHistos[ 'cov_uncorr_'+ivar ].GetBinContent(ibin,ibin)))
            uncerUnfoldHisto[ivar+'_CMInpStatTotal'].SetBinContent(ibin, ROOT.TMath.Sqrt( allHistos[ 'cov_uncorr_data_'+ivar ].GetBinContent(ibin,ibin)))
            uncerUnfoldHisto[ivar+'_StatTotal'].SetBinContent(ibin, orig_statunfunc)
            
            ratioHistos[ 'TotalUnc'+ivar ].SetBinContent( ibin, 1. )
            ratioHistos[ 'StatUnc'+ivar ].SetBinContent( ibin, 1. )
            ratioHistos[ 'SystUnc'+ivar ].SetBinContent( ibin, 1. )

            if norm_unc!=0:
                ratioHistos[ 'TotalUnc'+ivar ].SetBinError( ibin, ROOT.TMath.Sqrt(unc_tot**2+syst_tot**2)/norm_unc)
                ratioHistos[ 'StatUnc'+ivar ].SetBinError( ibin, orig_statunfunc/norm_unc)
                if len(sysUncert)>0: ratioHistos[ 'SystUnc'+ivar ].SetBinError( ibin, syst_tot/norm_unc)
            #print (norm_unc, ROOT.TMath.Sqrt(unc_tot**2+syst_tot**2 ), unc_tot, syst_tot, orig_statunfunc)
        
        #####################################

        ###### Plot unfolding results

        print ('|------> Drawing unfold plot:')
        drawUnfold(ivar=ivar, 
                   selection=sel, year=year,
                   process=process,
                   dataJetHisto=allHistos[ 'dataHistoGenBin' ].Clone(),
                   genJetHisto=signalHistos[ signalLabel+'_accepgen'+ivar+sel ].Clone(),
                   unfoldHisto=allHistos[ 'unfoldHisto'+ivar ].Clone(),
                   unfoldHistowoUnc=allHistos[ 'unfoldHistowoUnc'+ivar ].Clone(),
                   foldHisto=tunfolder.GetFoldedOutput("folded"+ivar).Clone(),
                   recoJetHisto=signalHistos[ signalLabel+'_reco'+ivar+'_nom'+sel+'_genBin' ].Clone(),
                   ratioUncHisto=ratioHistos[ 'StatUnc'+ivar ].Clone(),
                   ratiototUncHisto=ratioHistos[ 'TotalUnc'+ivar ].Clone(),
                   ratiosystUncHisto =ratioHistos[ 'SystUnc'+ivar ].Clone(),
                   altMCHisto =  altSignalHistos[altSignalLabel+'_accepgen'+ivar+sel],
                   labelX=variables[ivar]['label'],
                   maxX=variables[ivar]['bins'][-1],
                   tlegendAlignment=variables[ivar]['alignLeg'],
                   outputName=outputDir+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+signalLabel+(''.join(sysUncert))+'_Tunfold_'+version+'.'+ext
                   )

        ######### Plotting Uncertainties
        print ('|------> Drawing unfold uncertainty plot:')
        drawUncertainties(ivar, allHistos[ 'unfoldHisto'+ivar ],
                          uncerUnfoldHisto,
                          variables[ivar]['label'],
                          variables[ivar]['alignLeg'],
                          outputDir+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_UNC_'+version+'.'+ext,
                          year=year,
                        )
        
        ######### Plotting 2D matrices of various kinds
        print ('|------> Drawing various matrix:')
        draw2D( ivar,  tunfolder.GetRhoItotal("rhoI"+ivar, "Global correlations"), variables[ivar], outputLabel='data_rhoI', outputDir=outputDir)
        draw2D( ivar,  allHistos[ 'correlation_matrix_'+ivar ], variables[ivar], outputLabel='data_correlationMatrix', outputDir=outputDir)
        draw2D( ivar,  allHistos[ 'probaMatrix'+ivar ], variables[ivar], outputLabel='data_probaMatrix', outputDir=outputDir, addCorrelation=True, addCondition=True )
        draw2D( ivar, signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel], variables[ivar], outputLabel='data_respMatrix', outputDir=outputDir, addCorrelation=True )
        draw2D( ivar, allHistos[ 'cov'+ivar], variables[ivar], outputLabel='dataTotal_covMatrix', outputDir=outputDir, addCorrelation=True)
        draw2D( ivar, allHistos[ 'cov_uncorr_'+ivar], variables[ivar], outputLabel='uncorrUncRM_covMatrix', outputDir=outputDir, addCorrelation=True)
        draw2D( ivar, allHistos[ 'cov_uncorr_data_'+ivar], variables[ivar], outputLabel='dataInpStats_covMatrix', outputDir=outputDir, addCorrelation=True)
        draw2D( ivar, allHistos[ 'cov_uncorr_bkg_'+ivar], variables[ivar], outputLabel='BkgSubtractionSyst_covMatrix', outputDir=outputDir, addCorrelation=True)

        
        
        ######### Saving Histograms
        def renamingHistos( dictHistos ):
            for isam, ihis in dictHistos.items():
                ihis.SetName(isam)
                ihis.SetTitle(isam)
                ihis.Write()

        outputRootName = outputDir+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root'
        print ('|------> Saving histograms in rootfile: ', outputRootName)
        outputRoot = ROOT.TFile.Open( outputRootName, 'recreate' )
        renamingHistos( signalHistos )
        renamingHistos( sysSignalHistos )
        renamingHistos( altSignalHistos )
        #print ("adding varsighistos", varSignalHistos.items())
        renamingHistos( dataHistos )
        renamingHistos( bkgHistos )
        renamingHistos( allHistos )
        renamingHistos( uncerUnfoldHisto )
        tunfolder.Write()
        outputRoot.Close()

        print ('|------> Saving histograms in yodafile: ', outputRootName.replace('.root', '.yoda'))
        histToYoda = [  yoda.root.to_yoda( allHistos [ 'unfoldHisto'+ivar ] ) ]
        yoda.writeYODA( histToYoda, outputRootName.replace('.root', '.yoda') )


In [30]:
import yoda
from yoda import *

In [31]:
import gc

## All unfoldings

In [32]:
def loadFiles(year):
    
    #### Files
    dataFile = {}
    sigFiles = {}
    bkgFiles = {}
    sysSigFiles = {}
    lumi=0.    
    ### Lumi
   
    #### All data runs from single year files
    if year.startswith('all'):
        print('|------> Running combination of years')
        if not inputFolder:
            print('|------> For running all the data you must specify the input folder.\n Have a nice day :)')
            sys.exit(0)

        for ivar in variables.keys():
            print (inputFolder+'/'+selection.split('_')[1]+'/2017/Unfolding/'+ivar+'/'+('MCClosure' if 'Cross' in process else process )+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root')
            dataFile[ivar+'_2017'] = ROOT.TFile.Open( inputFolder+'/'+selection.split('_')[1]+'/2017/Unfolding/'+ivar+'/'+('MCClosure' if 'Cross' in process else process )+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
            dataFile[ivar+'_2018'] = ROOT.TFile.Open( inputFolder+'/'+selection.split('_')[1]+'/2018/Unfolding/'+ivar+'/'+('MCClosure' if 'Cross' in process else process )+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )

    #### Single year unfolding runs on skimmers
    else:
        print('|------> Running single year')
        
        dataFile['data'] = [ ROOT.TFile.Open(inputFolder+checkDict( ('JetHT' if selection.startswith("_dijet") else 'SingleMuon'), dictSamples )[year]['skimmerHisto']) ]
        
        for iy in ( ['2017', '2018'] if year.startswith('all') else [ year ] ):
            lumi = lumi + checkDict( ( 'JetHT' if selection.startswith('_dijet') else 'SingleMuon' ), dictSamples )[iy]['lumi']

        
        for isam in dictSamples:
            if 'MuEnriched' in isam: continue
            if not checkDict( isam, dictSamples )[year]['skimmerHisto'].endswith('root'): continue
            if isam.startswith( signalLabelBegin ) and not signalLabel.startswith((sysSignalLabelBegin,altSignalLabelBegin )):
                sigFiles[isam.split('_Tune')[0]] = [
                                ROOT.TFile.Open( inputFolder+checkDict( isam, dictSamples )[year]['skimmerHisto'] ),
                                checkDict( isam, dictSamples )
                            ]
            if isam.startswith( altSignalLabelBegin ) and not ( altSignalLabel.startswith( signalLabelBegin ) ):
                sigFiles[isam.split('_Tune')[0]] = [
                                ROOT.TFile.Open( inputFolder+checkDict( isam, dictSamples )[year]['skimmerHisto'] ),
                                checkDict( isam, dictSamples )]
            
            if isam.startswith(sysSignalLabelBegin):
                sysSigFiles[isam.split('_Tune')[0]] = [
                                    ROOT.TFile.Open( inputFolder+checkDict( isam, dictSamples )[year]['skimmerHisto'] ),
                                    checkDict( isam, dictSamples )
                            ] 
            
    return lumi, dataFile, sigFiles, bkgFiles, sysSigFiles

In [33]:
inputFolder = '../test/Samples/Dijet_V05/'
outputFolder = '../../../../../Results/Results_2022/'
main = 'HTbin'
alt = 'herwig'
process='data'

In [34]:
if main.startswith('Ptbin'):
    signalLabelBegin = 'QCD_Pt_'
    signalLabel = 'QCD_Pt_3200toInf'
elif main.startswith('HTbin'):
    signalLabelBegin = 'QCD_HT'
    signalLabel = 'QCD_HT2000toInf'
elif main.startswith('herwig'):
    signalLabelBegin = 'QCD_Pt-'
    signalLabel = 'QCD_Pt-150to3000'
if alt.startswith('Ptbin'):
    altSignalLabelBegin = 'QCD_Pt_'
    altSignalLabel = 'QCD_Pt_3200toInf'
elif alt.startswith('HTbin'):
    altSignalLabelBegin = 'QCD_HT'
    altSignalLabel = 'QCD_HT2000toInf'
elif alt.startswith('herwig'):
    altSignalLabelBegin = 'QCD_Pt-'
    altSignalLabel = 'QCD_Pt-150to3000'
    #altSignalLabel = 'QCD_Pt-15to7000'

sysSignalLabelBegin = 'sysQCD_' 
sysSignalLabels = ['sysQCD_puWeight_HT2000toInf',
                   'sysQCD_isrWeight_HT2000toInf',
                   'sysQCD_fsrWeight_HT2000toInf',
                   'sysQCD_jer_HT2000toInf',
                   'sysQCD_jesTotal_HT2000toInf',
                   'sysQCD_pdfWeight_HT2000toInf'
                   
                  ]


In [59]:
for selection in ['_dijetSel']:
    ##############PROBLEM with herwig 7 flatpower7 sample for 2017: it does not exist in v05, so using v04 herwig samples for this 
    print ("#######################")
    print ("#####%s selection#####"%((selection.split('_')[1])).split('Sel')[0])
    print ("#######################")
    
    variables = nSubVariables_dijetSel
    version = '_JanBkgSub_nominal'
    
    if selection.startswith('_W'): sysUnc = ['_jer', '_isrWeight', '_fsrWeight','_pdfWeight', '_puWeight', '_jesTotal', '_model', '_hdamp', '_Tune', '_erdON', '_CR', '_mtop']
    elif selection.startswith('_top'): sysUnc = ['_jer', '_isrWeight', '_fsrWeight','_pdfWeight', '_puWeight', '_jesTotal', '_model', '_hdamp', '_Tune', '_erdON', '_CR', '_mtop']
    elif selection.startswith('_dijet'): sysUnc = [ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal', '_model' ]
 
    
    for year in (['2017','2018']):
        lumi, dataFile, sigFiles, bkgFiles, sysSigFiles = loadFiles(year)
        print (dataFile.keys(), sigFiles.keys(),bkgFiles.keys(),sysSigFiles.keys())
        print ("#######################")
        print ("#####year=%s#####"%year)
        print ("#######################")
        
        print ("#######################")
        print ("####Selection=%s#####"%selection)
        print ("#######################")
        
        
        runTUnfold(dataFile=dataFile, sigFiles=sigFiles, bkgFiles=bkgFiles, sysSygFiles=sysSigFiles,
                       variables=variables, sel=selection, sysUncert=sysUnc, process='data' ,ext='pdf')
        
        #runTUnfold(dataFile=dataFile, sigFiles=sigFiles, altFiles=altSigFiles, varFiles=varSigFiles, bkg=bkgFiles, sysSigFiles=sysSigFiles, 
        #               variables=variables, sel=selection, sysUncert=sysUnc,process='data' ,ext='png')
        gc.collect()

#######################
#####dijet selection#####
#######################
|------> Running single year
dict_keys(['data']) dict_keys(['QCD_Pt-150to3000', 'QCD_HT100to200', 'QCD_HT200to300', 'QCD_HT300to500', 'QCD_HT500to700', 'QCD_HT700to1000', 'QCD_HT1000to1500', 'QCD_HT1500to2000', 'QCD_HT2000toInf']) dict_keys([]) dict_keys(['sysQCD_jer_HT100to200', 'sysQCD_jer_HT200to300', 'sysQCD_jer_HT300to500', 'sysQCD_jer_HT500to700', 'sysQCD_jer_HT700to1000', 'sysQCD_jer_HT1000to1500', 'sysQCD_jer_HT1500to2000', 'sysQCD_jer_HT2000toInf', 'sysQCD_jesTotal_HT100to200', 'sysQCD_jesTotal_HT200to300', 'sysQCD_jesTotal_HT300to500', 'sysQCD_jesTotal_HT500to700', 'sysQCD_jesTotal_HT700to1000', 'sysQCD_jesTotal_HT1000to1500', 'sysQCD_jesTotal_HT1500to2000', 'sysQCD_jesTotal_HT2000toInf', 'sysQCD_puWeight_HT100to200', 'sysQCD_puWeight_HT200to300', 'sysQCD_puWeight_HT300to500', 'sysQCD_puWeight_HT500to700', 'sysQCD_puWeight_HT700to1000', 'sysQCD_puWeight_HT1000to1500', 'sysQCD_puWeight_HT1500to2000', 'sy

|-----> Correlation:  0.9099701183001291
|-----> Condition Number:  25.71
|-----> Correlation:  0.8380588895752269
|-----> Correlation:  23.67558721973238
|-----> Correlation:  185.37413684320518
|-----> Correlation:  2.0266064165830824
|-----> Correlation:  1.876088548275885
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_0p5_3/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_0p5_3/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
|-------> Running single year 2017
|------> Unfolding Jet1_tau_0p5_4
|------> Cross check: plotting data vs all MC
Processing....... Jet1_tau_0p5_4_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_0p5_4_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simple

|------> Unfolding Jet1_tau_1_3
|------> Cross check: plotting data vs all MC
Processing....... Jet1_tau_1_3_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_1_3_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_1_3_fromData_QCD_HT2000toInf_TestProjectionReco_projectionRegular TrueReco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_1_3_fromData_QCD_HT2000toInf_TestDataBkgFakes_dataBkg+fakes_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_1_3_fromData_QCD_HT2000toInf_TestDataMinusBkgs_data-Bkgssignal true reco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
|------> Cross check: plotting response matrix for signal
|------> TUnfolding starts:
|------> TUnfolding adding input:
dict_keys(['allBkgHisto', 'allBkgHistoGenBin', 'allMCHisto', 'allMCHistoGen

|-----> Correlation:  0.0
|-----> Correlation:  434.6963458440176
|-----> Correlation:  2.537145340311147
|-----> Correlation:  2.5834849467219496
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_2_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_2_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
|-------> Running single year 2017
|------> Unfolding Jet1_tau_2_3
|------> Cross check: plotting data vs all MC
Processing....... Jet1_tau_2_3_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_2_3_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau_2_3_fromData_QCD_HT2000toInf_TestProjectionReco_projectionRegular 

|------> Unfolding Jet1_tau32
|------> Cross check: plotting data vs all MC
Processing....... Jet1_tau32_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau32_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau32_fromData_QCD_HT2000toInf_TestProjectionReco_projectionRegular TrueReco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau32_fromData_QCD_HT2000toInf_TestDataBkgFakes_dataBkg+fakes_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet1_tau32_fromData_QCD_HT2000toInf_TestDataMinusBkgs_data-Bkgssignal true reco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
|------> Cross check: plotting response matrix for signal
|------> TUnfolding starts:
|------> TUnfolding adding input:
dict_keys(['allBkgHisto', 'allBkgHistoGenBin', 'allMCHisto', 'allMCHistoGenBin', 'dataH

...

|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/2018/Unfolding/Jet2_tau_0p5_1/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
|-------> Running single year 2018
|------> Unfolding Jet2_tau_0p5_2
|------> Cross check: plotting data vs all MC
Processing....... Jet2_tau_0p5_2_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_0p5_2_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_0p5_2_fromData_QCD_HT2000toInf_TestProjectionReco_projectionRegular TrueReco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_0p5_2_fromData_QCD_HT2000toInf_TestDataBkgFakes_dataBkg+fakes_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_0p5_2_fromData_QCD_HT2000toInf_TestDataMinusBkgs_data-Bkgssignal true reco_simpleCompariso

|------> Unfolding Jet2_tau_1_1
|------> Cross check: plotting data vs all MC
Processing....... Jet2_tau_1_1_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_1_1_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_1_1_fromData_QCD_HT2000toInf_TestProjectionReco_projectionRegular TrueReco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_1_1_fromData_QCD_HT2000toInf_TestDataBkgFakes_dataBkg+fakes_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_1_1_fromData_QCD_HT2000toInf_TestDataMinusBkgs_data-Bkgssignal true reco_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
|------> Cross check: plotting response matrix for signal
|------> TUnfolding starts:
|------> TUnfolding adding input:
dict_keys(['allBkgHisto', 'allBkgHistoGenBin', 'allMCHisto', 'allMCHistoGen

|------> TUnfolding doUnfold:
|------> TUnfolding uncertainties:
|------> TUnfolding covariances
BC of unfHist systot+cov in quadrature tot from cov syst tot stat+unf unc from unfhisto
|------> Drawing unfold plot:
|------> Drawing unfold uncertainty plot:
unc. scale factor 2.0266952093857378e-08
|------> Drawing various matrix:
|-----> Correlation:  0.8359096218229429
|-----> Condition Number:  11.61
|-----> Correlation:  0.7841382122940583
|-----> Correlation:  6.258427335741469
|-----> Correlation:  203.39685409833373
|-----> Correlation:  4.7957710474191755
|-----> Correlation:  4.566851410906822
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/2018/Unfolding/Jet2_tau_1_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/2018/Unfolding/Jet2_tau_1_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
|-------> Running single

|-----> Correlation:  1828.9218614934016
|-----> Correlation:  6.271839741896092
|-----> Correlation:  11.5286184870491
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/2018/Unfolding/Jet2_tau_2_3/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/2018/Unfolding/Jet2_tau_2_3/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
|-------> Running single year 2018
|------> Unfolding Jet2_tau_2_4
|------> Cross check: plotting data vs all MC
Processing....... Jet2_tau_2_4_fromData_QCD_HT2000toInf_nom_dataallBkgs_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_2_4_fromData_QCD_HT2000toInf_TestProjectionGen_projectionRegular AccepGen_simpleComparisonPlot_dijetSel__JanBkgSub_nominal.png
Processing....... Jet2_tau_2_4_fromData_QCD_HT2000toInf_TestProjectionReco_projectionRegular TrueReco_simpleComparisonPl

Warning in <TH1F::Rebin>: Bin edge 20 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1F::Rebin>: Bin edge 20 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1F::Rebin>: Bin edge 20 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1F::Rebin>: Bin edge 20 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1F::Rebin>: Bin edge 20 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsisten

In [35]:
inputFolder = '../../../../../Results/Results_2022'
outputFolder = '../../../../../Results/Results_2022/'
main = 'HTbin'
alt = 'herwig'
process='data'

In [36]:
for selection in ['_dijetSel']:
    print ("#######################")
    print ("#####%s selection#####"%((selection.split('_')[1])).split('Sel')[0])
    print ("#######################")
    
    variables = nSubVariables_dijetSel
    version = '_JanBkgSub_nominal'
    
    if selection.startswith('_W'): sysUnc = ['_jer', '_isrWeight', '_fsrWeight','_pdfWeight', '_puWeight', '_jesTotal', '_model', '_hdamp', '_Tune', '_erdON', '_CR', '_mtop']
    elif selection.startswith('_top'): sysUnc = ['_jer', '_isrWeight', '_fsrWeight','_pdfWeight', '_puWeight', '_jesTotal', '_model', '_hdamp', '_Tune', '_erdON', '_CR', '_mtop']
    elif selection.startswith('_dijet'): sysUnc = [ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal', '_model' ]
 
    
    for year in (['all']):
        lumi, dataFile, sigFiles, bkgFiles, sysSigFile = loadFiles(year)
        #print (sigFiles.keys(),bkgFiles.keys(),dataFile.keys())
        print ("#######################")
        print ("#####year=%s#####"%year)
        print ("#######################")
        
        print ("#######################")
        print ("####Selection=%s#####"%selection)
        print ("#######################")
        
        
        runTUnfold(dataFile=dataFile, sigFiles={}, bkgFiles={}, sysSygFiles={},
                       variables=variables, sel=selection, sysUncert=sysUnc, process='data' ,ext='pdf')
        
        #runTUnfold(dataFile=dataFile, sigFiles=sigFiles, altFiles=altSigFiles, varFiles=varSigFiles, bkg=bkgFiles, sysSigFiles=sysSigFiles, 
        #               variables=variables, sel=selection, sysUncert=sysUnc,process='data' ,ext='png')
        gc.collect()

#######################
#####dijet selection#####
#######################
|------> Running combination of years
../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_0p5_1/data/outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_0p5_2/data/outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_0p5_3/data/outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_0p5_4/data/outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_1_1/data/outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
../../../../../Results/Results_2022/dijetSel/2017/Unfolding/Jet1_tau_1_2/data/outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
../../../../../Results/Resul

|-----> Correlation:  0.8641093884169565
|-----> Correlation:  9.486363077924553
|-----> Correlation:  138.09820480311316
|-----> Correlation:  1.7413118149021514
|-----> Correlation:  1.74603643341108
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_0p5_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_0p5_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet1_tau_0p5_3_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet1_tau_0p5_3_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet1_tau_0p5_3_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet1_tau_0p5_3_isrWeightDown_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet1_tau_0p5_3_fsrWeightUp_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet1_tau_0p5_3_fsrWeightDown_dijetSel
sysQCD_puWeight_HT2000toInf_reco

|------> Drawing unfold uncertainty plot:
unc. scale factor 2.1335729589242327e-08
|------> Drawing various matrix:
|-----> Correlation:  0.8789836941010368
|-----> Condition Number:  5.92
|-----> Correlation:  0.8454430794737098
|-----> Correlation:  0.0
|-----> Correlation:  0.0
|-----> Correlation:  1.7770139925099524
|-----> Correlation:  1.8147998531422722
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_1_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_1_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet1_tau_1_3_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet1_tau_1_3_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet1_tau_1_3_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet1_tau_1_3_isrWeightDown_dijetSel
sysQCD_fsrWeight_HT200

|------> Drawing unfold uncertainty plot:
unc. scale factor 1.004392288537479e-08
|------> Drawing various matrix:
|-----> Correlation:  0.8011890049804392
|-----> Condition Number:  5.64
|-----> Correlation:  0.7791845842870381
|-----> Correlation:  255.17783544645954
|-----> Correlation:  241.1694935113677
|-----> Correlation:  2.6017991789050194
|-----> Correlation:  3.013307373784354
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_2_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_2_2/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet1_tau_2_3_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet1_tau_2_3_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet1_tau_2_3_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet1_tau_2_3_isrWeightDown_dije

|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau32/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau32/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet2_tau_0p5_1_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet2_tau_0p5_1_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau_0p5_1_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau_0p5_1_isrWeightDown_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau_0p5_1_fsrWeightUp_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau_0p5_1_fsrWeightDown_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau_0p5_1_puWeightUp_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau_0p5_1_puWeightDown_dijetSel
sysQCD_pdfWeight_HT2000toInf_recoJet2_tau_0p5_1_pdfWeightUp_dijetSel
sysQCD_pdfWeight_HT2000toInf_recoJet2

|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet2_tau_0p5_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet2_tau_0p5_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet2_tau_1_1_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet2_tau_1_1_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau_1_1_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau_1_1_isrWeightDown_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau_1_1_fsrWeightUp_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau_1_1_fsrWeightDown_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau_1_1_puWeightUp_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau_1_1_puWeightDown_dijetSel
sysQCD_pdfWeight_HT2000toInf_recoJet2_tau_1_1_pdfWeightUp_dijetSel
sysQCD_pdfWeight_HT2000toInf_recoJet2_tau_1_1_p

|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet2_tau_1_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet2_tau_1_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet2_tau_2_1_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet2_tau_2_1_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau_2_1_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau_2_1_isrWeightDown_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau_2_1_fsrWeightUp_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau_2_1_fsrWeightDown_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau_2_1_puWeightUp_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau_2_1_puWeightDown_dijetSel
sysQCD_pdfWeight_HT2000toInf_recoJet2_tau_2_1_pdfWeightUp_dijetSel
sysQCD_pdfWeight_HT2000toInf_recoJet2_tau_2_1_pdfWe

|-----> Correlation:  0.7241336185298558
|-----> Correlation:  0.0
|-----> Correlation:  139.62385908907143
|-----> Correlation:  4.485601220356446
|-----> Correlation:  5.364977441735269
|------> Saving histograms in rootfile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet2_tau_2_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.root
|------> Saving histograms in yodafile:  ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet2_tau_2_4/data//outputHistograms_main_QCD_HT2000toInf_alt_QCD_Pt-150to3000.yoda
sysQCD_jer_HT2000toInf_recoJet2_tau21_jerUp_dijetSel
sysQCD_jer_HT2000toInf_recoJet2_tau21_jerDown_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau21_isrWeightUp_dijetSel
sysQCD_isrWeight_HT2000toInf_recoJet2_tau21_isrWeightDown_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau21_fsrWeightUp_dijetSel
sysQCD_fsrWeight_HT2000toInf_recoJet2_tau21_fsrWeightDown_dijetSel
sysQCD_puWeight_HT2000toInf_recoJet2_tau21_puWeightUp_dijetSel
sysQCD_puWe

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet1_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_0p5_1/data/Jet1_tau_0p5_1_fromData_QCD_HT2000toInf_dijetSel_responseMatrix_JanBkgSub_nominal.pdf has been created
Info in <TUnfold::SetConstraint>: fConstraint=0
Info in <TUnfold::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfold::TUnfold>: 20 input bins and 10 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:Accepted Gen Outer AK8 jet #tau_{1}^{(0.5)}[ufl])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #11 (xaxis:Accepted Gen Outer AK8 jet #tau_{1}^{(0.5)}[ofl])
Info in <TUnfoldDensity::RegularizeOneDistribution>: regularizing xaxis regMode=3 densityMode=1 axisSteering=*[UOB]
Info in <TCanvas::Print>: pdf file ../../../../../Results/Results_2022/dijetSel/all/Unfolding/Jet1_tau_0p5_1/data/Jet1_tau_0p5_1_fromData_sysQCD_jer_HT20